In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import pandas as pd
import numpy as np
from keras.models import load_model
import os
import time
from datetime import datetime
# CSV learning rate
from keras.callbacks import CSVLogger

# history
from keras.callbacks import History

In [ ]:
# To download dataset from google drive
!pip install gdown

In [ ]:
!gdown --folder https://drive.google.com/drive/folders/1K1Fg2AzMKnjcaMDHjvuhe0hs9tdpb3ZU?usp=sharing

In [ ]:
#Specify the number of features. w-HAR:120, UCI-HAR/UCI-HAPT:561, unimib:453, WISDM:405
featurenums = 405
#Specify the shape of 2D image that we will form. w-HAR:4x30, UCI-HAR/UCI-HAPT:33x17, unimib: 25*18, WISDM:27*15
img_rows = 27; img_cols = 15
#Number of activity labels, w-HAR:8, UCI-HAR:6, UCI-HAPT:12, unimib:9, WISDM:6
num_actions = 6
#Batch size, epochs, iterations for each CNN
batch_size = 128; epochs = 100; n_folds = 10
#Specify the number of taotal traning clusters.
trained_ucs = [1,2,3,4]

In [ ]:
startTime = datetime.now() # Get the start time of the code 
#For callbacks logs
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, epoch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, epoch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

In [ ]:
from keras.utils import np_utils

In [ ]:
# for trained_uc in trained_ucs:
#     #Specify output folder

# Specify Cluster
trained_uc = 1

output_path = 'CNN/WISDM_UC_k_means' + '/trained_with_user_' + str(trained_uc)
if not os.path.exists(output_path):
	os.makedirs(output_path)
	#load the features and the labels. 60% train, 20% xval, 20% test
df_features_train = pd.read_csv('WISDM_UC_k_means' + '/train_data_' + str(featurenums) + '_v1_uc_' + str(trained_uc) +'.csv', header=None)
df_labels_train = pd.read_csv('WISDM_UC_k_means' + '/train_labels_' + str(featurenums) + '_v1_uc_' + str(trained_uc) +'.csv', header=None)
		
df_features_test = pd.read_csv('WISDM_UC_k_means' +'/test_data_' + str(featurenums) + '_v1_uc_' + str(trained_uc) +'.csv', header=None)
df_labels_test = pd.read_csv('WISDM_UC_k_means' +'/test_labels_' + str(featurenums) + '_v1_uc_' + str(trained_uc) +'.csv', header=None)   
		
df_features_xval = pd.read_csv('WISDM_UC_k_means'+'/xval_data_' + str(featurenums) + '_v1_uc_' + str(trained_uc) +'.csv', header=None)
df_labels_xval = pd.read_csv('WISDM_UC_k_means'+'/xval_labels_' + str(featurenums) + '_v1_uc_' + str(trained_uc) +'.csv', header=None)  

# Get data for training and test
data_train = df_features_train.to_numpy()
labels_train = df_labels_train.to_numpy()
labels_train = labels_train - 1
labels_train = labels_train.ravel()
labels_orig_train = labels_train

# Extract test data
data_test = df_features_test.to_numpy()
labels_test = df_labels_test.to_numpy()
labels_test = labels_test - 1
labels_test = labels_test.ravel()
labels_orig_test = labels_test

# Extract xval data
data_xval = df_features_xval.to_numpy()
labels_xval = df_labels_xval.to_numpy()
labels_xval = labels_xval - 1
labels_xval = labels_xval.ravel()
labels_orig_xval = labels_xval

# Convert labels
labels_train = keras.utils.np_utils.to_categorical(labels_train, num_actions)
labels_test = keras.utils.np_utils.to_categorical(labels_test, num_actions)
labels_xval = keras.utils.np_utils.to_categorical(labels_xval, num_actions)

# 	#for cutting some of the features
# 	data_train = data_train[:,0:450]
# 	data_test = data_test[:,0:450]
# 	data_xval = data_xval[:,0:450]

# reshape the input 
data_train_shaped = data_train.reshape(data_train.shape[0], img_rows, img_cols,1)
data_test_shaped = data_test.reshape(data_test.shape[0], img_rows, img_cols,1)
data_xval_shaped = data_xval.reshape(data_xval.shape[0], img_rows, img_cols,1)
input_shape = (img_rows, img_cols,1)


x_train = data_train_shaped
y_train = labels_train
x_test = data_test_shaped
y_test = labels_test
x_xval = data_xval_shaped
y_xval = labels_xval


x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_xval = x_xval.astype('float32')


hist = History()
callbacks_list = [hist]
time_callback = TimeHistory()

In [ ]:
import tensorflow as tf 

In [ ]:
for i in range(0,n_folds):
	# compile the CNN
	model = Sequential()
	model.add(Conv2D(32, kernel_size=(1, 1),
						activation='relu',
						input_shape=input_shape))
	model.add(Conv2D(64, (3, 3), activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))#2,2
	model.add(Dropout(0.25))
	model.add(Flatten())
	model.add(Dense(128, activation='relu'))
	model.add(Dropout(0.5))
	model.add(Dense(num_actions, activation='softmax'))
	model.compile(loss=keras.losses.categorical_crossentropy,
					optimizer=tf.keras.optimizers.Adadelta(),
					metrics=['categorical_accuracy'])
	
			#Log for loss and accuracy
	filename_log = output_path + '/training_with_{0}_iter_{1}.log'.format(trained_uc,i)
	if (i != 0):
		del callbacks_list[-1]
	csv_logger = CSVLogger(filename_log)
	callbacks_list.append(csv_logger)
	
	
	
	history1 = model.fit(x_train, y_train,
				batch_size=batch_size,
				epochs=epochs,
				verbose=0,
				validation_data=(x_xval, y_xval), callbacks=[csv_logger, time_callback])
	score_train = model.evaluate(x_train, y_train,verbose = 1)
	print('train accuracy = ', score_train[1])
			#Log for training time
	time_log = output_path + '/training_time_with_{0}_iter_{1}.txt'.format(trained_uc,i)
	time_callback_list = time_callback.times
	np.savetxt(time_log, time_callback_list,fmt='%f1') 
	print("Executed code")
			#Save the model
	model.save(output_path + '/model_'+ str(featurenums) +'_train_with_user_' + str(trained_uc) + \
					'_iter_' + str(i))

del callbacks_list[-1]

44/44 [==============================] - 1s 12ms/step - loss: 1.0408 - categorical_accuracy: 0.6715
train accuracy =  0.6714593768119812
Executed code
44/44 [==============================] - 1s 12ms/step - loss: 1.0418 - categorical_accuracy: 0.6492
train accuracy =  0.6491732597351074
Executed code
44/44 [==============================] - 1s 12ms/step - loss: 0.9336 - categorical_accuracy: 0.6477
train accuracy =  0.6477354168891907
Executed code
44/44 [==============================] - 1s 12ms/step - loss: 1.1095 - categorical_accuracy: 0.6441
train accuracy =  0.644140899181366
Executed code
44/44 [==============================] - 1s 12ms/step - loss: 1.1116 - categorical_accuracy: 0.6528
train accuracy =  0.6527677774429321
Executed code
44/44 [==============================] - 1s 12ms/step - loss: 1.1579 - categorical_accuracy: 0.6664
train accuracy =  0.6664270162582397
Executed code
44/44 [==============================] - 1s 12ms/step - loss: 1.1609 - categorical_accuracy: 0.

In [ ]:
'''
doing main testing.
''' 
#Testing all UCs accuracy starts here
#UCs to be tested
tuned_ucs = [1,2,3,4] 
#Initialize the accuracy array
result_acc_baseline_all = np.ones((4,4))
result_acc_baseline_test = np.ones((4,4))	

In [ ]:
# for tuned_uc in tuned_ucs:
#     #load the data and labels. 

# load Cluster
tuned_uc = 1

df_features_test = pd.read_csv('WISDM_UC_k_means' +'/test_data_' + str(featurenums) + '_v1_uc_' + str(tuned_uc) +'.csv', header=None)
df_labels_test = pd.read_csv('WISDM_UC_k_means' + '/test_labels_' + str(featurenums) + '_v1_uc_' + str(tuned_uc) +'.csv', header=None)  
df_features_all = pd.read_csv('WISDM_UC_k_means' + '/data_' + str(featurenums) + '_v1_uc_' + str(tuned_uc) +'.csv', header=None)
df_labels_all = pd.read_csv('WISDM_UC_k_means' + '/labels_' + str(featurenums) + '_v1_uc_' + str(tuned_uc) +'.csv', header=None)	


# Extract test data
data_test = df_features_test.to_numpy()
labels_test = df_labels_test.to_numpy()
labels_test = labels_test - 1
labels_test = labels_test.ravel()
labels_orig_test = labels_test

# Extract all data
data_all = df_features_all.to_numpy()
labels_all = df_labels_all.to_numpy()
labels_all = labels_all - 1
labels_all = labels_all.ravel()
labels_orig_all = labels_all


# Convert labels
labels_test = keras.utils.np_utils.to_categorical(labels_test, num_actions)
labels_all = keras.utils.np_utils.to_categorical(labels_all, num_actions)

# 	#for cutting some of the features
# 	data_test = data_test[:,0:450]
# 	data_all = data_all[:,0:450]

# reshape the input 
data_test_shaped = data_test.reshape(data_test.shape[0], img_rows, img_cols,1)
data_all_shaped = data_all.reshape(data_all.shape[0], img_rows, img_cols,1)



x_test = data_test_shaped
y_test = labels_test
x_all = data_all_shaped
y_all = labels_all

x_test = x_test.astype('float32')
x_all = x_all.astype('float32')
	
	
# for trained_uc in trained_ucs:

# select trained Cluster
trained_uc = 1

result_acc_baseline_all_temp = []
result_acc_baseline_test_temp = []

for i in range(0,n_folds):
	#load the trained model
	model_path = 'CNN/WISDM_UC_k_means' + '/trained_with_user_'+ str(trained_uc)\
+'/model_'+ str(featurenums) +'_train_with_user_' + str(trained_uc) + '_iter_' + str(i)
	model = load_model(model_path)
				#testing
	score_baseline_test = model.evaluate(x_test,y_test,verbose=0)
	score_baseline_all = model.evaluate(x_all,y_all,verbose=0)
	result_acc_baseline_test_temp.append(score_baseline_test[1])
	result_acc_baseline_all_temp.append(score_baseline_all[1])

result_acc_baseline_test[trained_uc-1][tuned_uc-1] = np.mean(result_acc_baseline_test_temp)
result_acc_baseline_all[trained_uc-1][tuned_uc-1] = np.mean(result_acc_baseline_all_temp)

In [ ]:
#output the accuracy	
output_path = "CNN/WISDM_UC_k_means/Accuracy"
if not os.path.exists(output_path):
	os.makedirs(output_path)
output_filename = output_path + "/result_{0}_acc_baseline_test_UCs".format(featurenums)
np.save(output_filename + ".npy", result_acc_baseline_test)
np.savetxt(output_filename + ".txt", result_acc_baseline_test,fmt='%f1')	
output_filename = output_path + "/result_{0}_acc_baseline_all_UCs".format(featurenums)
np.save(output_filename + ".npy", result_acc_baseline_all)
np.savetxt(output_filename + ".txt", result_acc_baseline_all,fmt='%f1')	
	

In [ ]:
'''
doing fine tuning and testing.
'''  
#Specify the number of the last trainable layers. 1 means only train the last FC layer, 3 means only train the last 2 FC layers. We skipped 2 because the last second layer is just the dropout layer which doesn't affect the result.
layers_trainables = [1,3]
#initialize the array
result_acc_test = np.ones((4,4))

for layers_trainable in layers_trainables:
	for tuned_uc in tuned_ucs:
		
        #load features and labels.
		df_features_train = pd.read_csv('WISDM_UC_k_means' + '/train_data_' + str(featurenums) + '_v1_uc_' + str(tuned_uc) +'.csv', header=None)
		df_labels_train = pd.read_csv('WISDM_UC_k_means' + '/train_labels_' + str(featurenums) + '_v1_uc_' + str(tuned_uc) +'.csv', header=None)
				
		df_features_test = pd.read_csv('WISDM_UC_k_means' + '/test_data_' + str(featurenums) + '_v1_uc_' + str(tuned_uc) +'.csv', header=None)
		df_labels_test = pd.read_csv('WISDM_UC_k_means' + '/test_labels_' + str(featurenums) + '_v1_uc_' + str(tuned_uc) +'.csv', header=None)   
				
		df_features_xval = pd.read_csv('WISDM_UC_k_means' + '/xval_data_' + str(featurenums) + '_v1_uc_' + str(tuned_uc) +'.csv', header=None)
		df_labels_xval = pd.read_csv('WISDM_UC_k_means' + '/xval_labels_' + str(featurenums) + '_v1_uc_' + str(tuned_uc) +'.csv', header=None)  
		
		
		
		# Get data for training and test
		data_train = df_features_train.to_numpy()
		labels_train = df_labels_train.to_numpy()
		labels_train = labels_train - 1
		labels_train = labels_train.ravel()
		labels_orig_train = labels_train
		
		# Extract test data
		data_test = df_features_test.to_numpy()
		labels_test = df_labels_test.to_numpy()
		labels_test = labels_test - 1
		labels_test = labels_test.ravel()
		labels_orig_test = labels_test
		
		# Extract xval data
		data_xval = df_features_xval.to_numpy()
		labels_xval = df_labels_xval.to_numpy()
		labels_xval = labels_xval - 1
		labels_xval = labels_xval.ravel()
		labels_orig_xval = labels_xval
		

		# Convert labels
		labels_train = keras.utils.to_categorical(labels_train, num_actions)
		labels_test = keras.utils.to_categorical(labels_test, num_actions)
		labels_xval = keras.utils.to_categorical(labels_xval, num_actions)
		


# 		#for cutting some of the features
# 		data_train = data_train[:,0:450]
# 		data_test = data_test[:,0:450]
# 		data_xval = data_xval[:,0:450]

		# reshape the input 
		data_train_shaped = data_train.reshape(data_train.shape[0], img_rows, img_cols,1)
		data_test_shaped = data_test.reshape(data_test.shape[0], img_rows, img_cols,1)
		data_xval_shaped = data_xval.reshape(data_xval.shape[0], img_rows, img_cols,1)
		input_shape = (img_rows, img_cols,1)
		

		
		x_train = data_train_shaped
		y_train = labels_train
		x_test = data_test_shaped
		y_test = labels_test
		x_xval = data_xval_shaped
		y_xval = labels_xval
        
		x_train = x_train.astype('float32')
		x_test = x_test.astype('float32')
		x_xval = x_xval.astype('float32')
		

		
		for trained_uc in trained_ucs:
            #Set the output folder
			output_folder = 'CNN/WISDM_UC_k_means'  +'/trained_with_user_{0}/last{1}layer_trainable'.format(trained_uc, layers_trainable)
			if not os.path.exists(output_folder):
				os.makedirs(output_folder)
			hist = History()
			callbacks_list = [hist]
			time_callback = TimeHistory()
			result_acc_test_temp = []
			for i in range(0,n_folds):
	
				
				# Load the baseline CNN models
				trained_model_path = 'CNN/WISDM_UC_k_means/' + '/trained_with_user_{0}/model_{1}_train_with_user_{2}_iter_{3}'.format(trained_uc, featurenums, trained_uc,i)
				model = load_model(trained_model_path)
				
		
					
				# freeze all layers except the deeper layers
				for layer in model.layers[:-layers_trainable]:
					layer.trainable = False				
				model.compile(loss=keras.losses.categorical_crossentropy,\
				  optimizer=keras.optimizers.Adadelta(), metrics=['categorical_accuracy'])
								   
				# Print the trainable status
				for layer in model.layers:
					print(layer, layer.trainable)
				
				# Print model summary
				model.summary()    
				
				#Log for accuracy and Loss
				filename_log = output_folder + '/training_tune_with_user_{0}_iter_{1}_last{2}layer_trainable.log'.format(tuned_uc,i,layers_trainable)
				if (i != 0):
					del callbacks_list[-1]
				csv_logger = CSVLogger(filename_log)
				callbacks_list.append(csv_logger)
				
				history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=0,validation_data=(x_xval, y_xval), callbacks=[csv_logger, time_callback])
				score_train =model.evaluate(x_train, y_train, verbose=1)
				print('train accuracy is ', score_train[1])
				#Log for time
				time_log = output_folder + '/training_time_tune_with_user_{0}_iter_{1}_last{2}layer_trainable'.format(tuned_uc,i,layers_trainable)
				time_callback_list = time_callback.times
                #save the time log
				np.save(time_log + ".npy", time_callback_list)
				np.savetxt(time_log + ".txt", time_callback_list,fmt='%f1')
                #save the fine-tuned CNN model
				filename = output_folder + '/model_{0}_train_with_user_{1}_tune_with_user_{2}_iter_{3}_last{4}layer_trainable'\
					.format(featurenums, trained_uc, tuned_uc, i, layers_trainable)
				model.save(filename)
				
				score_test = model.evaluate(x_test, y_test, verbose=0)
				score_xval = model.evaluate(x_xval, y_xval, verbose = 0)
				result_acc_test_temp.append(score_test[1])
				print("Trained_uc =", trained_uc)
				print("Tuned_uc =", tuned_uc)
				print("Running Fold", i+1, "/", n_folds)
				print('Test accuracy:', score_test[1])
				print('Xval accuracy:', score_xval[1]) 
			del callbacks_list[-1]
			result_acc_test[trained_uc-1][tuned_uc-1] = np.mean(result_acc_test_temp)
	
				
	
	#Export the Accuracy	
	output_path = "CNN/WISDM_UC_k_means" + "/Accuracy"
	output_filename = output_path + "/result_{0}_acc_test_UCs_last{1}layer_trainable".format(featurenums, layers_trainable)
	np.save(output_filename + ".npy", result_acc_test)
	np.savetxt(output_filename + ".txt", result_acc_test,fmt='%f1')	

print('The entire code running time is ', datetime.now() - startTime)	